# Configuração do Ambiente

In [1]:
try:
  from sqlalchemy import ForeignKey, select, Column, Integer, Float, String, create_engine, func
  from sqlalchemy.ext.declarative import declarative_base
  from sqlalchemy.orm import sessionmaker, relationship
except:
  !pip install sqlalchemy
  from sqlalchemy import ForeignKey, select, Column, Integer, Float, String, create_engine, func
  from sqlalchemy.ext.declarative import declarative_base
  from sqlalchemy.orm import sessionmaker, relationship

In [2]:
Base = declarative_base()

<ipython-input-2-9bae6e3862ba>:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


# Criação das tabelas

In [3]:
class Client(Base):
    __tablename__ = 'client'
    # Atributos
    id = Column(Integer, primary_key=True)
    name = Column(String)
    cpf = Column(String(11), unique=True)
    address = Column(String)
    # Relacionamentos
    accounts = relationship("Account", back_populates="client")

    def __repr__(self):
      return f'User(id={self.id}, name={self.name}, cpf={self.cpf}, address={self.address})'

In [4]:
class Account(Base):
  __tablename__ = 'account'
  # Atributos
  id = Column(Integer, primary_key=True)
  kind = Column(String)
  agency = Column(String)
  num = Column(Integer, unique=True)
  saldo = Column(Float)
  client_id = Column(Integer, ForeignKey('client.id'))
  # Relacionamentos
  client = relationship("Client", back_populates="accounts")

  def __repr__(self):
    return f'Account(id={self.id}, kind={self.kind}, agency={self.agency}, num={self.num}, saldo={self.saldo})'

# Configurando conexão

In [5]:
engine = create_engine('sqlite://')

In [6]:
Base.metadata.create_all(engine)

# Povoamento das tabelas

In [7]:
Session = sessionmaker(bind=engine)

# Cria uma instância da sessão
with Session() as session:

    cliente = Client(
        name='Luiz',
        cpf='01295372351',
        address='rua x, num 25'
    )
    session.add(cliente)

    conta = Account(
        kind='Conta Corrente',
        agency='001',
        num=12345,
        saldo=540.99,
        client=cliente
    )
    session.add(conta)

    cliente = Client(
        name='Gustavo',
        cpf='01957193401',
        address='rua y, num 12A'
    )
    session.add(cliente)

    conta = Account(
        kind='Conta Poupança',
        agency='001',
        num=23121,
        saldo=1300.55,
        client=cliente
    )
    session.add(conta)

    conta = Account(
        kind='Conta Corrente',
        agency='001',
        num=23122,
        saldo=2000.51,
        client=cliente
    )
    session.add(conta)

    cliente = Client(
        name='Alan',
        cpf='91849185612',
        address='rua z, num 235'
    )
    session.add(cliente)

    conta = Account(
        kind='Conta Poupança',
        agency='001',
        num=12000,
        saldo=2015.50,
        client=cliente
    )
    session.add(conta)

    cliente = Client(
        name='Luiz',
        cpf='91751095422',
        address='rua w, num 33'
    )
    session.add(cliente)

    conta = Account(
        kind='Conta Poupança',
        agency='002',
        num=11551,
        saldo=8511.25,
        client=cliente
    )
    session.add(conta)

    conta = Account(
        kind='Conta Corrente',
        agency='002',
        num=18412,
        saldo=2015.92,
        client=cliente
    )
    session.add(conta)


    session.commit()

# Consultas

In [8]:
stmt = select(Client).where(Client.name.in_(['Luiz']))

print('Clientes com nome "Luiz":\n')

for user in session.scalars(stmt):
  print(user)

Clientes com nome "Luiz":

User(id=1, name=Luiz, cpf=01295372351, address=rua x, num 25)
User(id=4, name=Luiz, cpf=91751095422, address=rua w, num 33)


In [9]:
stmt = select(Client).order_by(Client.name.asc())

print('Registros ordenados pelo nome:\n')

for user in session.scalars(stmt):
  print(user)

Registros ordenados pelo nome:

User(id=3, name=Alan, cpf=91849185612, address=rua z, num 235)
User(id=2, name=Gustavo, cpf=01957193401, address=rua y, num 12A)
User(id=1, name=Luiz, cpf=01295372351, address=rua x, num 25)
User(id=4, name=Luiz, cpf=91751095422, address=rua w, num 33)


In [10]:
stmt_join = select(Client.name, Account.kind, Account.num, Account.saldo).join_from(Client, Account)

print('Junção das tabelas:\n')

for result in session.execute(stmt_join):
  print(result)

Junção das tabelas:

('Luiz', 'Conta Corrente', 12345, 540.99)
('Gustavo', 'Conta Poupança', 23121, 1300.55)
('Gustavo', 'Conta Corrente', 23122, 2000.51)
('Alan', 'Conta Poupança', 12000, 2015.5)
('Luiz', 'Conta Poupança', 11551, 8511.25)
('Luiz', 'Conta Corrente', 18412, 2015.92)


In [11]:
stmt_count = select(func.count('*')).select_from(Client)

print('Total de contas: ', end='')

for result in session.scalars(stmt_count):
  print(result)

Total de contas: 4


# Encerramento da conexão

In [12]:
session.close()